# APT Program information extractor

*Author: Rosa Diaz*

This notebook extracts from an \[apt_pid\].sql file information relevant to the execution of a program in any of the OTB exercises.

It extracts information from three different tables in the SQL file and atempts to create three different CSV files. 

To generate this file you should go the APT "file" menu and select the option "Export". Select the box "sql.file <span style="color:blue">\[*.sql*\]...</span>" and click  "OK". Save this file to your working directory.

# imports

* `numpy` is used to generate new colums for the DB table. Numpy is also the numerical computation package in Python and hence part of the engine under the hood of Pandas.
* `os` is used to change to the working directory
* `Astropy.table` is used to read the APT sql file data into a table
* `Pandas` is used to convert the astropy data to DB format and manipulate the data to extract only the needed information
* `csv` is used to save the results in a *csv* format file
* `apt_sql` Class that reads the SQL input format file into a Numpy table. This piece of the code was written by Jeff Valenti.

In [1]:
import pandas as pd
import numpy as np
import os
import csv
from astropy.table import Table
import sys
import matplotlib.pyplot as plt
import apt_sql as asql
#matplotlib.style.use('ggplot')

As a first step we will try to determine if we have the latests versions of Pandas and Numpy

In [2]:
# Get Version information
print("Pandas version: {0}".format(pd.__version__),'\n')
print("Numpy version: {0}".format(np.__version__),'\n')
print("Python version : {0}".format(sys.version_info[0]),'\n')

Pandas version: 0.25.0 

Numpy version: 1.14.2 

Python version : 3 



## Introduction

This scrip reads the SQL files the Astronomer Proposal Tool (APT) generates as input to the Planing and Proposing System Data Base (PPSDB). These files are generated with the APT tool using the `Export` to sql option in the `File` APT menu. This scrip reads information from the **visit** and **exposures** tables and extract relevant information to understand what data will be obtained during the OTB tests runs. 

This script uses Jeff Valenti's code (https://github.com/spacetelescope/jwst-apt-tools) to extract the information from the APT SQL files. This is then used to create a CVS file with a subset of the data. The ultimate result are spreadsheets that are provided to the Test Coordinator and DMS to be able to track all the data that will be generated with the OTB or EMTB test runs.  

There is only one type of files we will use throughout this notebook. These are APT oput SQL files with names like:

    623.sql

All the files should be in the same working directory. 

In [3]:
# For the purpose of visualizing the data ub the tables, 
# we will set up the script to render the output in the Notebook interactively usng Ipython.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Define and access working directory

We set the path to our working directory and check it exists.
 

In [5]:
# Set Working Directory 
#os.chdir("/grp/jwst/dms2/testdata/E2E/GSEG3EMTBDR1/")
#os.chdir("/my_working/area/path/goes/here/")
work_dir = "GSEG3EMTBDR2"
os.chdir("/grp/jwst/dms2/testdata/E2E/GSEG3/"+work_dir+"/")
print("My new working directory:\n" + os.getcwd())
%pwd

My new working directory:
/grp/jwst/dms2/testdata/E2E/GSEG3/GSEG3EMTBDR2


'/grp/jwst/dms2/testdata/E2E/GSEG3/GSEG3EMTBDR2'

#### Define the output files 
Set output file to append the information. Three files will be created:
* visits_csv_file2.csv file with the information relevant to the vist
* exposures_csv_file2.csv with the information per exposure
* merged_csv_file2.csv is an atempt to merge the information from both files. It does not work well yet.


In [6]:

f1=open(work_dir+'_visits_csv_file_v1.csv', 'w')
f2=open(work_dir+'_exposures_csv_file_v1.csv', 'w')
f3=open(work_dir+'_merged_csv_file_v1.csv', 'w')
file_duration=open(work_dir+'_file_durations_test_v6.csv',"w+")
line=" Program,"+'observations,'+' total charged duration,'+' total charged duration hours\n'
file_duration.write(line)

76

Set the max printing rows to be 10, so I don't overwhelm the printed workbooks.

In [7]:
pd.options.display.max_rows = 10

## Define structured data to extract

The data we want to extract requires to read the apropiate SQL table and stract only the observations of interest; usually a subset of all the available observations in the SQL file. The APT sql files are usually named by the user; in here we assume these have name format *pid.sql*; where *pid* is the proposal ID number. 

The information to extract is provided in a dictionary with elements `key=order number` and `values= [array]`.  
* `order number` refers to the order in which the observation from a given proposal should be executed in the OTB exercise. 
* `[array]` is an array of values with first element equal to the PID and the rest with all the obs # to execute within a single OP (Observation Plan). All the observations should be from the same instrument.

For example, we want Obs 13 and 14 of PID 619 to be in the same OP that will run during Normal Ops 6. We also want for this OP to be the third one to run. The `key:array` combination will be: `3:[619,13,14]`.

NOTE: in the box below I have the order from previous runs. 

In [8]:
# list of proposals and observations to extract from the SQL file.
# Format: oder:[PID,obs1,obs2,obs3,...]

# GSEG3 Dry run # 2 EMTB January 6 2020
#621: FGS, 618: NIRISS, 617: NIRCam, 619: MIRI, 620: NIRSpec, 622: WFSC, 631: OSS RTCP Certification
prog_obs_list={1:[633,17,18,19,20,21,22,23,24],2:[633,25],3:[621,101,102,103,104,105],4:[621,93,94,95,96],
        5:[621,107,108,109,110,111,112],6:[621,97,98,99,100],7:[618,13],8:[618,14],9:[631,21,22,23,24,25],
        10:[619,51],11:[619,52,53,54,55,56,57], 12:[620,13], 13:[620,14], 
        14:[617,125,126,127,128,129,130,131,132,133,134], 15:[617,135], 16:[622,17], 17:[622,18]}

#Normal Ops 7 OTB Decembe 16 2019
#prog_obs_list={1:[630,6], 2:[624,14,15], 3:[628,37,38], 4:[637,5], 5:[721,18], 6:[623,52], 
#               7:[625,42], 8:[643,8,9], 9:[625,38,42], 10:[625,49,50], 11:[632,6], 
#               12:[643,5], 13:[626,34,36], 14:[623,47,48,49,50,51], 15:[628,39,40,41], 
#              16:[628,42,43,44], 17:[623,46], 18:[629,18,19,20,21,22,23,24,25,26,27], 
#               19:[624,16,17], 20:[721,17], 21:[626,37], 22:[625,39,40,41,43,44,45], 
#               23:[729,30,31], 24:[625,47,48,46], 25:[626,38,39], 26:[720,37,38,39], 
#               27:[721,16], 28:[643,7], 29:[643,6], 30:[632,7]}

#prog_obs_list={1:[632,6], 2:[624,14,15], 3:[628,37,38], 4:[721,18], 5:[623,52], 6:[625,42], 
#               7:[625,49,50], 8:[655,1], 9:[632,7], 10:[643,8,9,11], 
#               11:[629,18,19,20,21,22,23,24,25,26,27], 12:[643,5], 13:[626,34,36], 
#               14:[623,47,48,49,50,51], 15:[637,5], 16:[628,39,40,41,42,43,44], 17:[623,46], 
#               18:[624,16,17], 19:[721,17], 20:[626,37], 21:[625,39,40,41,43,44,45], 
#               22:[729,30,31], 23:[625,47,48,46], 24:[626,38,39], 25:[720,37,38,39], 
#               26:[625,38], 27:[643,3], 28:[643,6,7,10],29:[721,16],}

#621: FGS, 618: NIRISS, 617: NIRCam, 619: MIRI, 620: NIRSpec, 622: WFSC, 631: OSS RTCP Certification
#prog_obs_list={1:[633,12,13,14,15],2:[633,16],3:[621,81,82,83,84,85,86],4:[621,73,74,75,76],
#        5:[621,87,88,89,90,91,92],6:[621,77,78,79,80],7:[618,11],8:[618,12],9:[631,16,17,18,19,20],
#        10:[619,44],11:[619,45,46,47,48,49,50], 12:[620,9], 13:[620,10], 
#        14:[617,114,115,116,117,118,119,120,121,122,123], 15:[617,124], 16:[622,15], 17:[622,16]}
#prog_obs_list={2:[619,44],1:[633,12,13]}

#GSEG3 Dry Run 1 EMTB October 14 2019
#621: FGS, 618: NIRISS, 617: NIRCam, 619: MIRI, 620: NIRSpec, 622: WFSC, 631: OSS RTCP Certification
#prog_obs_list={1:[633,12,13,14,15],2:[633,16],3:[621,81,82,83,84,85,86],4:[621,73,74,75,76],
#        5:[621,87,88,89,90,91,92],6:[621,77,78,79,80],7:[618,11],8:[618,12],9:[631,16,17,18,19,20],
#       10:[619,44],11:[619,45,46,47,48,49,50], 12:[620,9], 13:[620,10], 
#        14:[617,114,115,116,117,118,119,120,121,122,123], 15:[617,124], 16:[622,15], 17:[622,16]}
#prog_obs_list={2:[619,44],1:[633,12,13]}

############################
# From PREVIOUS RUNS       #
############################
# Normal Ops 6
#prog_obs_list= {1:[630,5],2:[624,7,9],3:[739,12],4:[624,12],5:[623,35,36,42,44],
#          6:[623,45],7:[626,31,32],8:[625,29],9:[625,35,36],10:[632,4],11:[721,14],
#          12:[623,43],13:[643,1,2],14:[625,30,31,32],15:[729,22],16:[623,37,38,39,40],
#          17:[643,3,4],18:[625,33],19:[624,8],20:[729,27,23,24],21:[729,25,26,28,29],22:[632,4],
#          23:[736,23,24,25,26,27,28,29,30,31,32,33,34,35],24:[733,22,23,24,25,26,27],
#          25:[738,9,10,11,12,13,14,15,16],
#          26:[739,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]}

# Normal Ops 5
#prog_obs_list={1:[630,4],2:[721,11,12],3:[623,26,27],4:[626,22,28,29,27,25],5:[632,3],6:[625,23],
#        7:[729,20,21],8:[628,26,27,28],9:[623,2],10:[623,28],11:[625,24],12:[623,32,33],
#        13:[625,27,28],14:[729,19],15:[628,34,35,36,32,33],16:[623,29,30],17:[625,25],18:[625,26],
#        19:[623,31],20:[720,34,35,36],21:[623,34],22:[632,3],23:[721,13]}

#GSEG3 Dry Run # 1 
#621: FGS, 618: NIRISS, 617: NIRCam, 619: MIRI, 620: NIRSpec, 622: WFSC, 631: OSS RTCP Certification
#prog_obs_list={1:[621,61,62,63,64,65,66],2:[621,53,54,55,56],3:[621,67,68,69,70,71,72],4:[621,57,58,59,60],
#        5:[618,7],6:[618,8],7:[619,31,32,33,34,35],8:[619,37,38,39,40,41], 9:[620,7], 10:[620,8], 
#        11:[617,103,104,105,106,107,108,109,110,111,112], 12:[617,113], 13:[622,13], 14:[622,14], 
#        15:[631,11,12,13,14,15] }

# SOC4.3
#prog_obs_list={1:[99506,1,3],2:[99504,1,2],3:[99503,1,2],4:[99501,1,2,3],
#        5:[99502,1,2,3,4],6:[99508,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38],
#       7:[99507,1,3,4],8:[99505,1,2],9:[99506,1,3],10:[99504,1,2]}

#Normal Ops 4
#prog_obs_list={1:[630,3],2:[628,22.23.24],10:[721,10,9,7,8],15:[620,11],14:[626,15],5:[626,16],
#        7:[626,14],3:[793,21,22,23,24,25],8:[626,17]}

# Since we can input the information in any order, we need to sort the table.
order_run=dict(sorted(prog_obs_list.items()))

#number of OPs we will request
n_ops=len(prog_obs_list.keys())


### The SQL Files 

The SQL file includes information for the following tables:

* apt_diagnostics
* association_candidates
* exposures
* fixed_target
* instruments
* link
* link_requirements
* mosaic
* moving_target
* observation
* observation_association_candidates
* program
* special_requirements
* target
* target_description
* visit
* visit_linkspec
* visit_overheads
* visit_targets
* visit_track

This script uses only a subset of these tables.

#### Instrument specific files:

The SQL files includes some tables that will change depending on the instuments used in the APT file. For example, if the APT files include only MIRI observations, the table in the SQL file will be called something like *miri_exposure_specification*. Here the tables produced for MIRI. Again, we use only a subset of these tables.

* miri_exposure_specification
* miri_imaging_dither
* miri_lrs_dither
* miri_mrs_dither
* miri_target_acq
* miri_templates

## Extraction of data

We will use this dictionary to loop over all the proposals and observations we want to extract, and saves the information in the CSV file. Note that for each proposal we provided the Obs number and the order in which these should be executed during the OTB run. This also will be the order in which these will be added to the CSV file.


In [9]:
# The script will create a list of OPs to run in the order indicated by 
# the dictionary keys.

# To simplify our life it will calculate the overall duration
overall_duration=0

# The size of the dictionary declared before provides with the number of OPs we will 
# request. Remember that the order is provided by the keys in the dictionary.
for p in range(n_ops):
    exspec=0 # Set to zero when the exp_spec table does not exist
    expos =0 # Set to zero when the exposures table does not exist
    for k in order_run.keys():
        if p+1 == k:  
            # Since the firs element in the array defined above is the PID, we use
            # this information to identify the SQL file to read
            sql_file=str(prog_obs_list[k][0])+'.sql'
            pid=prog_obs_list[k][0]
            order=k

    sql=asql.Sqlfile(sql_file) #Reads the SQL file for that proposal

    # It then creates a Pandas DataFrame with the "instruments" table within the SQL file.
    # This table includes  observation number, prime science instrument,  program number,
    # record_id, and visit number
    table_inst=sql.table('instruments',browser=False).to_pandas()
    
    # Given that an APT file can include many instruments, we need to identify the instrument 
    # that is used by all the observations listed in each key. Remember that each element of 
    # the dictionary defined above should have observations for a single instrument.
    obs1=prog_obs_list[p+1][1]
    t0=table_inst.loc[(table_inst['program'] == pid) & (table_inst['observation'] == obs1),
                      ["prime_si"]]
    iinst=t0.iloc[0]
    
    # We then can use the instrument information to identify the exposure_sepcification table 
    # for that proposal; which needs to include the instrument name.
    exp_spec=iinst[0].lower()+'_exposure_specification'

    #FGS is special case for which we will benefit from knowing inforamtion about the Detector
    # used; so in this case we also need to read the "templates" table.
    if iinst[0] == 'FGS':
        templates=iinst[0].lower()+'_templates'
        table_templates = sql.table(templates, browser=False).to_pandas()
    
    # We then convert three tables (read by the J. Valenti's code as numpy tables) to Pandas
    # DataFrames. The tables we convert are: [instrument]_exposure_specification, visit, and 
    # exposures
    
    tablenames=sql.tablenames
    if exp_spec in tablenames:
        table_exp_spec = sql.table(exp_spec, browser=False).to_pandas()
        exspec=1
    else:
        print("Table %s does not exist" % exp_spec)
    table_visits = sql.table('visit', browser=False).to_pandas()
    if 'exposures' in tablenames:
        table_exps=sql.table('exposures',browser=False).to_pandas()
        expos=1
    else:
        print("Table exposures does not exist")
    
    # We also attempts to merge the information in the visit and exposures tables.
    # This is not working as expected yet but the code is here.
    if expos == 1:
        merged_table0=pd.merge(table_visits,table_exps, on=['program','observation','visit'], left_on=None, right_on=None,
           left_index=False, right_index=False, sort=True, suffixes=('_1', '_2'), copy=True, indicator=False,
           validate="1:m")
    
        # Again, we also attempt to merges the information in the visit file and the esposure specificaition 
        # table. This is not working as expected yet.
        if exspec == 1:
             merged_table1=pd.merge(table_exp_spec,table_visits, on=['program','observation','visit'], left_on=None, right_on=None,
               left_index=False, right_index=False, sort=False, suffixes=('_3', '_4'), copy=True, indicator=False,
               validate=None)
        else:
             merged_table1 = merged_table0
    else:
        mergerd_table1 = table_visits
        
    #list(merged_table1.columns.values)
    Header1=True
    Header2=True
    Header3=True
    
    ############################################################################
    # Now the code loops through all the observations we want to extract.
    ############################################################################
    index1=0
    sum_all_vals=0
    obs_str=''

    for obs in prog_obs_list[p+1][1:]:
        index1 = index1 + 1
        
        print(pid, ' observation=',obs1)
        # It extracts only certain columns from the exposures table in a particular order to
        # be the one we want in our output file.
        t2=table_exps.loc[(table_exps['program'] == pid) & (table_exps['observation'] == obs),
               ["program","observation","visit","AperName","apt_label",'data_volume_mib', 
                'dither_point_index', 'exposure', 'exposure_duration', 'exposure_spec_order_number',
                'external_parallel_slot_duration', 'mosaic_tile_order_number', 'number_of_groups',
                'number_of_integrations', 'number_of_scas', 'mosaic_tile_order_number', 
                'number_of_groups','number_of_integrations','number_of_scas','mosaic_tile_order_number', 
                'number_of_groups','number_of_integrations', 'number_of_scas', 
                'photon_collecting_duration', 'pointing_sequence_number','pointing_type', 'readout_pattern', 
                'target_id', 'target_order_number']]

        
        #Since we want to include the priority order in our output, 
        # we add a column to our DataFrame with this information.
        nrows=t2.shape[0]
        newcol=[]
        for i in range(nrows):
            newcol.append(str(order))
        t2.insert(1, "priority", newcol, True)
        
        #We also extract only certain columns from the visits table and save them in
        # a particular order to be the one we want in our output file.
        t1=table_visits.loc[(table_visits['program'] == pid) & (table_visits['observation'] == obs),
                            ["program","observation","visit","template","time_series_observation",
                             "charged_duration","overhead_duration","scheduling_duration",
                             "science_duration","guide_star_acquisition_duration","number_of_scas",
                             "slew_duration","data_volume","mosaic","visit_type"]]
        
        #We also add a column with the priority or order to be executed in the exposures
        # table.
        nrows=t1.shape[0]
        newcol=[]
        for i in range(nrows):
            newcol.append(str(order))
        t1.insert(1, "priority", newcol, True)
        
    
        # In order to simplify our life and make sure we fill all the time we have 
        # within a test, we calculate the total charged duration for a given OP as well
        # as the total duration for the whole exercise (i.e. the sum of all OPs).
        #
        #Pandas has the option to sums all values for all the rows so we use this to
        # add all rows with the same observation number; though, we are only interested in the
        # field "charged_duration"
        sumvals1=t1.loc[(table_visits['program'] == pid) & (table_visits['observation'] == obs)].sum(axis = 0, skipna = True)
        
        sum_all_vals += sumvals1.charged_duration # adds together all the charged durations for each observation
        overall_duration += sumvals1.charged_duration # adds together all the charged durations for each program
        
        # We will print this sum in the CSV file at the end of each OP section.
        obs_str = obs_str.strip() + str(t1['observation'].values[0])+' - '


        #In here we try to extract certain columns from the merged table; but again this is
        # not producing the desaired results. The problem here is that some of the field names
        # for the same type of information can change from one table to the other and from one
        # instrument to the other. So the merge results in multiple rows when we should have only
        # one row. The solution would be for APT to have the same field names for the same information.
        t3=merged_table1.loc[(merged_table1['program'] == pid) & (merged_table1['observation'] == obs),
             ["program","observation","visit","visit_type","template","time_series_observation","filter","pupil",
              'subarray',"mosaic",'readout_pattern','number_of_groups','number_of_groups_long','number_of_groups_short',
              'number_of_integrations','number_of_integrations_long','number_of_integrations_short','number_of_scas',
              "pcs_mode","special_commanding","charged_duration","overhead_duration","scheduling_duration","science_duration",
              "guide_star_acquisition_duration",'data_volume']]
        
        #We add to this DataFrame a coulum with the priority or order to be executed
        nrows=t3.shape[0]
        newcol=[]
        for i in range(nrows):
            newcol.append(str(order))
        t3.insert(1, "priority", newcol, True)

        
        # If the instrument is FGS, it will extract the detector information. But we don't use it.
        if iinst[0] == 'FGS':
            t4=table_templates.loc[(table_templates['program'] == pid) & (table_templates['observation'] == obs),
                                   ["program","observation","detector"]]
            
        #appends the information in table t1, t2, and t3 to separate CVS files
        if Header1:
            t1.to_csv(f1, sep='\t')
            Header1 = False
        else:
            t1.to_csv(f1, sep='\t',  header=None)
        
        if Header2:
            t2.to_csv(f2, sep='\t')
            Header2 = False
        else:
            t2.to_csv(f2, sep='\t',  header=None)
        #t2.to_csv(f2, sep='\t')
        if Header3:
            t3.to_csv(f3, sep='\t')
            Header3 = False
        else:
            t3.to_csv(f3, sep='\t',  header=None)
            
    #sumvals=t1.loc[(table_visits['program'] == pid) & (table_visits['observation'] == obs)].sum(axis = 0, skipna = True)
    
    # After each OP we will add a row that has the total charged duration.
    data=[['-','-',obs_str,'-','-','Total program (hr)',"{:.2f}".format(sum_all_vals/3600),'Added total',"{:.2f}".format(overall_duration/3600),'-','-','-','-','-','-'],
          ['.','.','.','.','.','.','.','.','.','.','.','.','.','.','.']]
    df = pd.DataFrame(data, columns = ["program","observation","visit","template","time_series_observation",
                             "charged_duration","overhead_duration","scheduling_duration",
                             "science_duration","guide_star_acquisition_duration","number_of_scas",
                             "slew_duration","data_volume","mosaic","visit_type"]) 
        
    
    # Once we have the data for all the observations for an OP, we append the total charged 
    #duration information to the CSV file.
    df.to_csv(f1, sep='\t',  header=None)
    line=str(t1['program'].values[0])+', '+obs_str.strip()+', '+str(sum_all_vals)+', '+str(sum_all_vals/3600)+', '+str(overall_duration)+'\n'
    file_duration.write(line)


633  observation= 17
633  observation= 17
633  observation= 17
633  observation= 17
633  observation= 17
633  observation= 17
633  observation= 17
633  observation= 17


/Users/rdiaz/anaconda3/envs/astroconda/lib/python3.5/site-packages/pandas/core/indexing.py:1404: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


70

633  observation= 25


42

...

45

622  observation= 17


44

622  observation= 18


31

In [10]:
# We close all the files

f1.close()
f2.close()
f3.close()
file_duration.close()
print('done')

done
